# 《数据采集与清洗》
## 第一次作业内容： 网页抓取
### 具体目标：
+ 用``urllib.requests``库发起一次``get``请求，输出``response``文本信息；
+ 用``Requests``库伪装成火狐浏览器发起一次``post``请求，输出``response``文本信息；
+ 用``Requests``库中的``session``对象发出``get``请求，设置``cookies``并获取，输出获取的``cookies``内容；
+ 实现抓取网页的不去重深度遍历算法，自选合适的种子网站和相关参数，输出结果；
+ 编写抓取网页的广度遍历算法（含去重和不去重），自选合适的种子网站和相关参数，输出结果；
+ (选)将抓取网页的去重深度遍历算法封装成对象（类），并测试。

### 注：
+ 代码要有注释，结果要有分析；
+ 本次作业提交截至时间：2020年3月10日(星期二)；
+ 文件命名规则: 班级号+学号+姓名+作业序号，示例：``1_20188989899_张三_1``；
+ 提交方式：1班发送至邮箱：632994085@qq.com；2班发送至邮箱：786888939@qq.com.

### 问题1：用``urllib.requests``库发起一次``get``请求，输出``response``文本信息。

In [12]:
# 这里编写代码
import urllib.request
data = bytes(urllib.parse.urlencode({'word': 'hello'}),encoding='utf8')              # 相当于一个将{'word': 'hello'}转变为计算机可识别语言的步骤
response = urllib.request.urlopen('http://httpbin.org/post',data=data)               # 用urlopen发送请求
print(response.read())

b'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {\n    "word": "hello"\n  }, \n  "headers": {\n    "Accept-Encoding": "identity", \n    "Content-Length": "10", \n    "Content-Type": "application/x-www-form-urlencoded", \n    "Host": "httpbin.org", \n    "User-Agent": "Python-urllib/3.7", \n    "X-Amzn-Trace-Id": "Root=1-5e6636a6-3c574db3c6be4677cf3a0d19"\n  }, \n  "json": null, \n  "origin": "58.44.244.108", \n  "url": "http://httpbin.org/post"\n}\n'


### 这里对结果进行分析。


本来我测试的是网址https://www.cnblogs.com/vamei/archive/2012/05/29/2524376.html
但是得出来的反馈太长了，里面还没有我认识的，所以就换成了ppt上的例子


+ headers            头
+ Accept-Encoding       可用的压缩方式      identity
+ Content-Length        消息体长度          10
+ Content-Type         消息体类型         application/x-www-form-urlencoded
+ host              主机             httpbin.org
+ User-Agent          客户端身份          Python-urllib/3.7
+ X-Amzn-Trace-Id      唯一识别代码        Root=1-5e6631a7-c53d164851146c308058bb20
+ url              统一地址           http://httpbin.org/post"\n}\n

### 用``Requests``库伪装成火狐浏览器发起一次``post``请求，输出``response``文本信息。

In [13]:
# 这里编写代码
from urllib import request,parse

url = "http://httpbin.org/post"
headers = {"User-Agent":'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)',"host":'httpbin.org'}# 用火狐浏览器进行伪装（客户端身份）
dict = {"name":"Hackdata"}                                                                    # form改变
data = bytes(parse.urlencode(dict),encoding="utf8")
req = request.Request(url=url,data=data,headers=headers,method="POST")                        # 用Request发送定制请求
response = request.urlopen(req)
print(response.read().decode("utf-8"))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "name": "Hackdata"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "13", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)", 
    "X-Amzn-Trace-Id": "Root=1-5e6636aa-f6b1c998e34b7860c489f018"
  }, 
  "json": null, 
  "origin": "58.44.244.108", 
  "url": "http://httpbin.org/post"
}



### 这里对结果进行分析。

与上面的大致变化如下：

1. 最后输出print(response.read().decode("utf-8"))使得反馈信息\n直接转移成了换行符。
2. form从字典{'word': 'hello'}变成了字典{"name": "Hackdata"}
3. 消息体长度从10变成了13
4. 因为进行了火狐浏览器伪装，所以客户端身份User-Agent的python3.7发生了改变
5. 唯一标识发生了改变

### 用``Requests``库中的``session``对象发出``get``请求，设置``cookies``并获取，输出获取的``cookies``内容。 

In [22]:
# 这里编写代码
import requests

s = requests.Session()                                                    # 创建一个session对象 
s.get('http://httpbin.org/cookies/set/sessioncookie/123456789')           # 用session对象发出get请求，设置cookies
r = s.get("http://httpbin.org/cookies")                                   # 用session对象发出另外一个get请求，获取cookies
r.text                                                                    # 显示结果

'{\n  "cookies": {\n    "sessioncookie": "123456789"\n  }\n}\n'

### 这里对结果进行分析。
对步骤的分析已经放在了代码注释中

### 实现抓取网页的不去重深度遍历算法，自选合适的种子网站和相关参数，输出结果。


In [28]:
# 这里编写代码

import requests, re

count = 3                                                                      # 爬取次数（层数）
r = re.compile(r'href=[\'"]?(http[^\'" >]+)')
seed = 'http://cookdata.cn/course/course_introduction/1/'                      # 选择要爬取的种子网站
queue = [seed]                                                                 # 设置一个队列，用以存储接下来要爬取的网站的地址
used = set()                                                                   # 设置一个集合，保存已经抓取过的 URL
storage = {}                                      

while len(queue) > 0 and count > 0:                                            # 队列不空且爬取次数不为零（按要求还要继续爬时）
    try:
        url = queue.pop(0)                                                     # 得到队列中的第一个网站
        html = requests.get(url).text                                          # 发送get请求，并得到消息体
        storage[url] = html                    
        used.add(url)                                                          # 将已经抓取过的URL存入used集合中
        new_urls = r.findall(html)                                             # re的方法，将该网站上所有的有关于http开头的url找到并作为列表存储
        for new_url in new_urls:                                               # 此循环将未抓取的URL添加到queue中
            if new_url not in queue:                                           # 没有去重，去重的代码在这里if里面加XXXXX not in used and XXXXXX
                queue.append(new_url)
        count -= 1
    except Exception as e:
        print(url)
        print(e)
        count -= 1


### 这里对结果进行分析。
对步骤的分析已经放在了代码的注释中

Failed to establish a new connection 无法建立新连接 Errno 11002

### 编写抓取网页的广度遍历算法（含去重和不去重），自选合适的种子网站和相关参数，输出结果。 

In [25]:
# 这里编写代码
# 这里写的是没有去重的遍历算法，去重的只要在if条件语句里面稍微变化一下就行了，略




import requests, re

init_url = "https://cn.hao123.com/"
r = re.compile(r'href=[\'"]?(http[^\'" >]+)')
queue = [init_url]
used = []
maxnum = 5

while len(queue) > 0 and maxnum > 0:
    try:
        url = queue.pop(0)
        html = requests.get(url).text
        used.append(url)                                                       # 将已经抓取过的URL存入used集合中
        new_urls = r.findall(html)                                             # 找到接下来要爬的网站
        for new_url in new_urls:
            if new_url not in queue:
                queue.append(new_url)
        maxnum -= 1
    except Exception as e:
        print(url)
        print(e)
        maxnum -= 1

### 这里对结果进行分析。
无

### (选)将抓取网页的去重深度遍历算法封装成对象（类），并测试。 

In [23]:
# 这里编写代码
import requests, re

class Crawler_URL:
    
    def __init__(self,url):                             # 相当于一个初始化的操作
        self.url = url
        
    def Get_All_URL(self,count):                        # 定义类下面的一个方法，用以爬取种子网站的相关网站
        r = re.compile(r'href=[\'"]?(http[^\'" >]+)')
        queue = [self.url]
        used = set()
        
        while len(queue) > 0 and maxnum > 0:
            try:
                url = queue.pop(0)
                html = requests.get(url).text
                used.add(url)
                new_urls = r.findall(html)
                for new_url in new_urls:
                    if new_url not in used and new_url not in queue:
                        queue.append(new_url)
                maxnum -= 1
            except Exception as e:
                print(url)
                print(e)
                maxnum -= 1
        return list(used)

In [24]:
# 测试
url = Crawler_URL("http://cookdata.cn/course/course_introduction/1/")
url.Get_All_URL

<bound method Crawler_URL.Get_All_URL of <__main__.Crawler_URL object at 0x000002361D281F60>>

定义一个Crawler_URL的类，用以创建Crawler_URL的对象。
Crawler_URL的对象下有Get_All_URL的方法。